In [1]:
import os
os.chdir('../..')
from pipelines.util import *
import pandas as pd
from datetime import datetime
from collections import Counter
import re

In [2]:
data = pd.read_csv(os.path.join(WDIR, 'true-north/true_north_may_2024_clean.csv'))

In [3]:
summary = data.describe()
summary.loc['top_percent_of_count'] = (summary.loc['freq'] * 100 / summary.loc['count']).astype(float).round(1)

In [4]:
summary

,last_updated,Create Date,Do you feel the True North report identified the key challenges and opportunities facing the region?,Are you interested in attending future True North events?,Are you currently a B Corp or in the process of becoming a B Corp?,Would you be interested in hearing more from Brabners about the B Corp process?,Company name,City,Industry,sector,location,Which theme of the True North report do you most identify with and could support activity around?,How would you like to be involved with the True North network?,company_size
count,181,181,181,180,179,179,169,136,127,180,44,181,44,44
unique,170,154,2,2,2,2,162,70,50,146,17,13,7,7
top,2022-03-28 00:29:00,2022-03-28 00:29:00,Yes,Yes,No,No,Go-International,London,Professional Training & Coaching,Legal,Lancashire,"People, skills and the future",Keeping informed about the latest True North n...,0-9
freq,7,21,174,178,155,96,2,19,9,6,9,56,17,20
top_percent_of_count,3.9,11.6,96.1,98.9,86.6,53.6,1.2,14.0,7.1,3.3,20.5,30.9,38.6,45.5


In [5]:
company_names = pd.Series(data['Company name'].sort_values())
print(company_names.unique())

['2030hub' 'AHR' 'Abeceder' 'Access Creative College'
 'Accountable Recruitment' 'Advanced Manufacturing Research Centre'
 'Agent Marketing Ltd' 'Agile Automations' 'Armstrong Watson LLP' 'Azets'
 'B-Engineering Group' 'BHP LLP'
 'Bank of England (Agency for the North West)' 'Barton Legal Limited'
 'Bicical' 'Big Hand 4 Business' 'Blackpool Council' 'Bruntwood'
 'Business Doctors' 'Business Health Institute' 'Business of Science'
 'CBI' 'Carbon Happy World' 'Cazenove Capital'
 'Chris Shakespeare Coaching' 'Codurance' 'Cortus Advisory Group Limited'
 'Coverdale Barclay' 'DES Group UK' 'Daneel Williams' 'Deloitte LLP'
 'E-Rail' 'ESGmark' 'Elevategm' 'Endless LLP' 'Essential-Sourcing' 'F-LEX'
 'Farrans (Construction) Limited' 'Finance Yorkshire' 'Fleetsolve Ltd'
 'Fort Search' 'Fox Brothers' 'Fragomen' 'Freshfield'
 'Giles Metcalfe Digital' 'Go-International' 'Gripple Limited'
 'Group Rhodes' 'Groupsignature' 'HSBC' 'Handelsbanken'
 'Harrowells Limited' 'Hawthorne & Burman' 'Headsrenewabl

There is LLoyds banking group and lloyds bank plc - we're assuming these are different companies.

In [6]:
# convert the month column to a datetime object
data['month'] = pd.to_datetime(data['Create Date'])

#convert the item to a formatted value in yyyy-mm format.
data['month_formatted'] = data['month'].apply(datetime.strftime, format='%Y-%m').sort_index(ascending=True)

In [7]:
# take the number of members and calculate the number that joined each month.
monthly_members = pd.DataFrame(data['month_formatted'].value_counts(ascending=False)).reset_index()

In [8]:
monthly_orgs = data.drop_duplicates(subset='Company name', keep='last')
monthly_orgs = pd.DataFrame(monthly_orgs['month_formatted'].value_counts(ascending=False)).reset_index()

In [9]:
def decimal_date(data):
    # make a unix timestamp column
    data['timestamp'] = pd.to_datetime(data['month_formatted'], format='%Y-%m').astype(int) / 10**9
    # make a decimal date and round to 2dp.
    data['year'] = data['timestamp'].div((86400*365.25)).add(1970).round(2)
    # drop the timestamp column
    data.drop(columns='timestamp', inplace=True) 
    # set year and formatted month as the index so they aren't included in the cumsum.
    data.set_index(['year', 'month_formatted'], inplace=True, append=True)

    return data

In [10]:
def calculate_cumsum(data, count_name):
    # order by date, then do the cumsum. reset the index, drop the original index column as not needed
    data = pd.DataFrame(data.sort_index(level=2).cumsum().reset_index().drop(columns='level_0'))
    data.rename(columns={'count': f'{count_name}'}, inplace=True)
    return data

In [11]:
# apply above functions to data
cs_monthly_members = calculate_cumsum(decimal_date(monthly_members), count_name='individuals')
cs_monthly_orgs = calculate_cumsum(decimal_date(monthly_orgs), count_name='orgs')

In [12]:
# merged the two dataframes
cs_merged = cs_monthly_members.merge(cs_monthly_orgs, how='inner', on=['year', 'month_formatted'])

In [13]:
# write to file
cs_merged.to_csv(os.path.join(SRC_DIR,'overview/membership/_data/cumsum.csv'), index=False)

Calculating some summary stats

In [14]:
total_members = len(data.index)

total_companies = summary.loc['unique', 'Company name']

top_company_size = summary.loc['top', 'company_size']

top_company_size_pct = summary.loc['top_percent_of_count', 'company_size']

top_industry = summary.loc['top', 'Industry']

top_industry_pct = summary.loc['top_percent_of_count', 'Industry']


### Word frequency in the sector column

In [15]:
def normalize_string(s):
    # Convert to lowercase and remove non-alphanumeric characters (keeping spaces)
    try:
        s = s.lower()
    except:
        print(f"'{s}' is not a stirng type. Converting to string\n")
        return str(s)
    s = re.sub(r'[^a-z0-9\s]', '', s)
    return s
data['normalized_sector'] = data['sector'].apply(normalize_string).str.split(';')
 
# Flatten the list and further split by spaces to handle multi-word strings
all_words = [word for sublist in data['normalized_sector'] for item in sublist for word in item.split()]
 
# Count the occurrences of each word
word_counts = Counter(all_words)
 
# Find the most common word
most_common_words = word_counts.most_common(10)
least_common_words = word_counts.most_common()

banned_words = ['and', 'or', 'of', 'it', 'the', 'for', 'with', 'we']
# print("The 10 most common words, not including 'and' are:") 
# for word, count in most_common_words:
#     if word == banned_words:
#         continue
#     print(f"'{word}' with {count} occurences")

print("The most common words, with at least 2 occurences are:")
words = []
counts = [] 
for word, count in least_common_words:
    if word in banned_words:
        continue
    if count >= 10:
        print(f"'{word}' with {count} occurences")
        words.append(word)
        counts.append(count)

sector_counts = pd.DataFrame(data={'name': words, 'count': counts}).set_index('name')
# sector_counts['colour'] = round((sector_counts['count'] - min(sector_counts['count'])) / sector_counts['count'].max(), 3)
sector_counts.to_csv(os.path.join(SRC_DIR, 'overview/membership/_data/sector_word_counts.csv'))
# sector_strings = [item for sublist in data['sector'].str.split(';') for item in sublist]
# string_counts = Counter(sector_strings)
# print(string_counts.most_common())


'nan' is not a stirng type. Converting to string

The most common words, with at least 2 occurences are:
'services' with 20 occurences
'construction' with 18 occurences
'education' with 16 occurences
'manufacturing' with 15 occurences
'finance' with 13 occurences
'professional' with 13 occurences
'engineering' with 13 occurences
'technology' with 12 occurences
'business' with 11 occurences
'development' with 10 occurences


In [16]:
advisory_council = 11

if summary.loc['top', "Are you currently a B Corp or in the process of becoming a B Corp?"] == 'No':
    b_corps_pct = round(100 - summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"], 1)
else:
    summary.loc['top_percent_of_count', "Are you currently a B Corp or in the process of becoming a B Corp?"]

membership_increase = cs_merged['individuals'].pct_change().mul(100).iloc[-1].round(1)

northern_stars = len(pd.read_csv(os.path.join(SRC_DIR, 'overview/insights/_data/northern_stars.csv')))

names = ["Total members", 
         "Total companies", 
         "Membership increase", 
         f"Companies with {top_company_size} employees", 
         "Top industry", 
         "Geographic reach", 
         "Advisory council", 
         "Northern stars", 
         "B Corps"]

values = [total_members, 
          total_companies, 
          membership_increase, 
          top_company_size_pct, 
          top_industry_pct, 
          "4", 
          advisory_council, 
          northern_stars, 
          b_corps_pct]

footnotes = ["People", 
             "Unique companies", 
             "Since last month", 
             "Of members work in companies of this size", 
             f"Of members work in {top_industry}", 
             "placeholder", 
             "Members represent the network on the True North advisory council", 
             "Companies have been featured as Northern Stars", 
             "Of member's organisations are B Corps or are joining"]

posts = ['','','%','%','%','','','','%']

urls = ['/overview/membership', '/overview/membership', '/overview/membership', '', '', '', 'https://www.brabners.com/insights/true-north/true-north-advisory-council-launches', '/overview/insights', '/themes/sustainable-growth/b-corporations']
dashboard = pd.DataFrame(data={'name':names, 'value': values, 'footnote': footnotes, 'post': posts, 'url': urls})

dashboard

,name,value,footnote,post,url
0,Total members,181,People,,/overview/membership
1,Total companies,162,Unique companies,,/overview/membership
2,Membership increase,1.1,Since last month,%,/overview/membership
3,Companies with 0-9 employees,45.5,Of members work in companies of this size,%,
4,Top industry,7.1,Of members work in Professional Training & Coa...,%,
5,Geographic reach,4,placeholder,,
6,Advisory council,11,Members represent the network on the True Nort...,,https://www.brabners.com/insights/true-north/t...
7,Northern stars,5,Companies have been featured as Northern Stars,,/overview/insights
8,B Corps,13.4,Of member's organisations are B Corps or are j...,%,/themes/sustainable-growth/b-corporations


In [17]:
dashboard.to_csv(os.path.join(SRC_DIR, '_data/dashboard/true_north_members_list.csv'), index=False)